In [13]:
import numpy as np
import math
from decimal import Decimal, getcontext
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from math import factorial as f
import scipy.linalg
from tqdm import tqdm
import plotly.io as pio

# , 2.000, 2.129, 3.132, 3.223, 5.250, 3.587, 4.787
# , 0.790, 0.594, 0.330, 0.248, 0.324, 0.334, 0.459

('0.790,', '0.594,', '0.330,', '0.248,', '0.324,', '0.334,', '0.459')

In [14]:
# Set the precision you need
getcontext().prec = 32

### Input Parameters

In [15]:
def read_model_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # Read the first line to find out which model user wants to access
    target_model = lines[0].split(": ")[1].strip()
    model_name = None
    velocities = []
    thicknesses = []
    found_model = False
    for i, line in enumerate(lines[1:], 1):
        line = line.strip()
        if line == target_model:
            found_model = True
        elif found_model:
            model_name = lines[i].split(": ")[1].strip()
            velocities = list(map(float, lines[i+1].split("[")[1].split("]")[0].split(",")))
            thicknesses = list(map(float, lines[i+2].split("[")[1].split("]")[0].split(",")))
            
            velocities = np.array([Decimal(i) for i in velocities])
            thicknesses = np.array([Decimal(i) for i in thicknesses])
            delta_t = thicknesses/velocities
            break

    return model_name, velocities, thicknesses, delta_t

file_path = r'input.txt'
Model_Name, v, d, delta_t = read_model_data(file_path)

v = np.array([Decimal(i) for i in v])
d = np.array([Decimal(i) for i in d])
delta_t = np.array([Decimal(i) for i in delta_t])

##### Compute J Value

In [16]:
from math import factorial as f
def J_value(J):
    Jj = Decimal(0.0)
    filename = "J_values.dat"
    with open(filename, 'w') as file:
        for j in range(J+1):
            if j == 0:
                Jj = Decimal(1.0)
            else:
                num = Decimal(2 * j) * Decimal(f(2 * j))
                den = Decimal((2 * j - 1)) * Decimal(4 ** j) * Decimal(f(j)) ** 2
                Jj = num / den
            file.write(f"J({j}) = {Jj}\n")
    return Jj

##### Compute V2j value

In [18]:
def V2j(j, v, delta_t):
    """
    Sample V2j for an input j value. (eqn 4.8)

    Args:
        j(int) : input j value
        v (1D array): velocity array for an input layered earth model
        delta_t (1D array): one-way vertical travel time for each layer

    Returns:
        V2j (Decimal object)
    """
    v2j = np.zeros(j+1, dtype=object)
    for jj in range(1, j+1):
        N = len(v)
        sum1 = sum(Decimal(v[k])**(2*jj) * Decimal(delta_t[k]) for k in range(N))
        T0 = 2 * sum(Decimal(delta_t[k]) for k in range(N))
        v2j[jj] = (2 * sum1) / T0
    # Write Jj value to the .dat file
    filename = "v2j_Values.dat"
    with open(filename, 'w') as file:  
        for jj in range(1,j+1):
            file.write(f"v2j({jj}) = {Decimal(v2j[jj]):.32E}\n")  
    return v2j


### Compute Taylor Coeffecients(Cn)

#### compute matrix A

In [155]:
def compute_A(n, v, delta_t):
    """
    Generate the A matrix with 1st and each even column storing the corresponding A2nj values for n=1, 2, 4, 6,...
    (eqn. 4.12)
    Args:
        n(int) : the number of taylor coeffecients
        v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
        delta_t (float): Array containing the one-way vertical traveltimes of a horizontally isotropic layered Earth.

    Returns:
        A (2D array) It returns a sparse matrix with necessary values stored at 1st, 2nd, 4th, ..., 2*nth columns
    """
    # Initiate matrix A with zeros
    A = np.zeros((n+1, 2*n+1), dtype=object)
    
    # Numbers of layers
    N = len(v)

    # Compute A1j 
    for j in tqdm(range(1, n+1), desc = 'A1j'):
        V2j_T0 = 2 * sum(Decimal(v[k]**(2*j)) * Decimal(delta_t[k]) for k in range(N))  # eqn 4.8
        Jj = J_value(j)  # eqn 4.7
        A[j, 1] = V2j_T0 * Jj

    # Compute A2j
    for j in tqdm(range(1, n+1), desc = 'A2j'):
        A[j, 2] = sum(A[mu, 1] * A[j-mu+1, 1] for mu in range(1, j+1))  # eqn 4.10
        
    # Compute for all columns (Aj2n)
    m = n
    for nn in tqdm(range(2, (2*n+1)//2 + 1), desc = 'A2nj'):
        A2n_2j = A[:, 2*nn-2]
        A2j = A[:, 2]
        m -= 1
        for j in range(1, m+1):
            A[j, 2*nn] = sum(A2n_2j[mu] * A2j[j-mu+1] for mu in range(1, j+1))  # eqn 4.12
                
    return A

#### compute matrix B

In [156]:
def compute_B(n, v, delta_t):
    """
    Generate the B matrix. 
    Args:
        n(int) : the number of taylor coeffecients
        v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
        delta_t (float): Array containing the one-way vertical traveltimes of a horizontally isotropic layered Earth.

    Returns:
        B (2D array) It returns a matrix with 
        0th column is all zeros,
        1st column corresponds to B1j,
        2nd column corresponds to B2j.
    """
    B = np.zeros((n+1, 3), dtype=object)
    # compute B1j for j=0
    B[0,1] = Decimal(2 * sum(delta_t[k] for k in range(len(delta_t))))  #eqn. 4.14
    # compute B1j for all values of j
    for j in range(1, n+1):
        V2j_T0 = 2 * sum(Decimal(v[k]**(2*j)) * Decimal(delta_t[k]) for k in range(len(v)) )
        B[j, 1] = V2j_T0 * J_value(j+1)  # eqn 4.14
    
    # compute B2j for j=0
    B[0, 2] = B[0,1]*B[0,1]    
    # compute B2j for all values of j
    for j in range(1, n+1):
        B2j = sum(B[mu, 1] * B[j-mu, 1] for mu in range(j+1))  # eqn 4.16
        B[j, 2] = B2j
    
    return B

#### compute matrix C

In [157]:
def compute_C(n, A, B):
    """
    Calculate the Taylor coeffecients upto input n values. 
    Args:
        n(int) : the number of taylor coeffecients
        v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
        delta_t (float): Array containing the one-way vertical traveltimes of a horizontally isotropic layered Earth.

    Returns:
        C (2D array) : It returns a matrix with 
        0th column stores all Taylor coeffecients,
        1st column stores their log values.
    """
    C = np.zeros((n+1, 2), dtype=object)  # Use dtype=object to store Decimal objects
    # Compute C0
    C[0, 0] = B[0, 2]
    C[0, 1] = C[0, 0].log10()
    # Compute C1
    C[1, 0] = B[1, 2] / A[1, 2]
    C[1, 1] = C[1, 0].log10()
    # Compute C2 to Cn
    for nn in tqdm(range(2, n+1), desc = 'Cn'):
        sum_term = sum(C[nn-j, 0] * A[j+1, 2*(nn-j)] for j in range(1, nn))
        C[nn, 0] = ((B[nn, 2]) - sum_term) / A[1, 2*nn]
        C[nn, 1] = abs(C[nn, 0]).log10()

    # Write A matrix to the .dat file
    filename = "C_matrix.dat"
    with open(filename, 'w') as file:
        file.write("Sl No.\t\tCn\t\t\t\t\t\tlog(Cn)\n")
        file.write("-----------------------------------------------------------------------------------------------------------\n")
        for idx, row in enumerate(range(n), start=1):
            file.write(f"{idx}\t\t" + '\t\t'.join(f"{C[row, col]:.32E}" for col in range(2)) + '\n')
    return C

In [158]:
'''compute Taylor coeffecints for n values.'''
n = 100
A = compute_A(n, v, delta_t)
B = compute_B(n, v, delta_t)
C = compute_C(n, A, B) 

Cn: 100%|██████████| 99/99 [00:00<00:00, 8231.30it/s]


In [159]:
# check for A103, A45
A103 = A[1, 8]*A[3, 2] + A[2, 8]*A[2, 2] + A[3, 8]*A[1, 2]
print("A103, A[3, 10]:",A103, A[3, 10])
A45 = A[1, 2]*A[5,2] + A[2, 2]*A[4,2] + A[3, 2]*A[3,2] + A[4, 2]*A[2,2] + A[5, 2]*A[1,2] 
print("A45, A[5, 4] :",A45, A[5, 4],"\n")

# check for B23
B23 = B[0,1]*B[3,1] + 2*B[1,1]*B[2,1] + B[3, 1]*B[0,1]
print("B23, B[3, 2] :", B[3, 2], B23, "\n")

# check for C4, C5
C4 = (B[4, 2] - C[3, 0]*A[2, 6] - C[2, 0]*A[3, 4] - C[1, 0]*A[4, 2])/A[1, 8]
print("C4, C[4, 0]:",C[4, 0], C4)
C5 = (B[5, 2] - C[4, 0]*A[2, 8]- C[3, 0]*A[3, 6] - C[2, 0]*A[4, 4] - C[1, 0]*A[5, 2])/A[1, 10]
print("C5, C[5, 0] ",C5, C[5, 0])
C6 = (B[6, 2] - C[5, 0]*A[2, 10]- C[4, 0]*A[3, 8] - C[3, 0]*A[4, 6] - C[2, 0]*A[5, 4] - C[1, 0]*A[6, 2])/A[1, 12]
print("C6, C[6, 0] ",C6, C[6, 0])
C7 = (B[7, 2] - C[6, 0]*A[2, 12]- C[5, 0]*A[3, 10] - C[4, 0]*A[4, 8] - C[3, 0]*A[5, 6] - C[2, 0]*A[6, 4]- C[1, 0]*A[7, 2])/A[1, 14]
print("C7, C[7, 0] ",C7, C[7, 0])

A103, A[3, 10]: 82205767532115382.759816762424191 82205767532115382.759816762424191
A45, A[5, 4] : 101318243454.31710986397658251284 101318243454.31710986397658251284 

B23, B[3, 2] : 13410.738028926107537835945549259 13410.738028926107537835945549259 

C4, C[4, 0]: 1.4515257387231693239248867902673E-8 1.4515257387231693239248867902850E-8
C5, C[5, 0]  1.6182988333286344661353249321025E-10 1.6182988333286344661353249321025E-10
C6, C[6, 0]  -4.2323726880758345779530638762033E-13 -4.2323726880758345779530638836559E-13
C7, C[7, 0]  -4.8009988682545062668308463577810E-14 -4.8009988682545062668308463577810E-14


### Compute incidence angles until critical angle

In [160]:
def compute_incidence_angles(v, incident_angle, increment):
    """
    Generate an array of incidence angles for a given velocity model. 
    Args:
        v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
        incident_angle(float) : initial incident angle 
        increment (float): Increment in the incident angle.

    Returns:
        incidence_angles(1D array)
        crit_angle(float) : critical angle in degrees
    """
    crit_angle = Decimal(math.degrees(math.asin(v[0]/v.max())))
    # incidence_angles = np.arange(incident_angle, crit_angle, increment)
    incidence_angles = []
    
    angle = incident_angle
    while angle <= crit_angle:
        incidence_angles.append(angle)
        increment=increment/Decimal(1.006)
        angle = angle + increment

    return incidence_angles, crit_angle

incident_angle = Decimal(1.0)
increment = Decimal(0.1)
incidence_angles, crit_angle = compute_incidence_angles(v, incident_angle, increment)
print(f"critical angle in radians : {crit_angle}")
print(f"critical angle in radians : {math.asin(v[0]/v.max())}")

critical angle in radians : 17.62929083896010951093558105640113353729248046875
critical angle in radians : 0.30768916993152734


### Compute offsets (eqn 4.1)

In [161]:
def compute_offsets(v, delta_t, incidence_angles):
    """
    Calculates an array of offsets for input array of incidence angles and velocity model. 
    Args:
        v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
        delta_t (1D array): Array containing the one-way vertical traveltimes of a horizontally isotropic layered Earth.
        incidence_angles(1D array) : Array of incidence angles
        
    Returns:
        offsets (1D array): Array of offsets for input array of incidence angles and velocity model.
    """
    Xp = np.zeros(len(incidence_angles), dtype = object)
    for theta in range(len(incidence_angles)):
        p = Decimal(np.sin(math.radians(incidence_angles[theta])))/v[0]
        for k in range(len(v)):
            num = 2 * (v[k]**2) * (delta_t[k]) * p
            den = (1 - (v[k]**2) * (p**2)).sqrt()
            Xp[theta] += num / den
    return Xp

Xp = compute_offsets(v, delta_t, incidence_angles)


### Compute Analytical travel time (eqn 4.2)

In [162]:
def compute_two_way_traveltimes(v, delta_t, incident_angle):
    """
    Compute two-way traveltimes for a given set of incident angles and velocities.

    Parameters:
        v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
        delta_t (1D array): Array containing the one-way vertical traveltimes of a horizontally isotropic layered Earth.
        incidence_angles(1D array) : Array of incidence angles

    Returns:
        Tp (1D array): Array of two-way travel times for input array of incidence angles and velocity model.
    """
    Tp = np.zeros(len(incidence_angles), dtype = object)
    for theta in range(len(incidence_angles)):
        p = Decimal(np.sin(math.radians(incidence_angles[theta])))/v[0]
        for k in range(len(v)):
            num = 2 * delta_t[k]
            den = np.sqrt(1 - (v[k]**2) * (p**2))
            Tp[theta] += num / den
    return Tp

Tp = compute_two_way_traveltimes(v, delta_t, incident_angle)

### Taylor Series Approximation of Squared Two-Way Traveltime Offset Relation 

In [163]:
def T_squared_taylor(approximation_order, Xp, C):
    """
    Compute the Taylor approximation of the squared traveltimes for a set of offsets of a given velocity model.
    Approximation order specifies the degree of Taylor appproximation of time-offset series.

    Parameters:
    approximation_order (int): The order of the Taylor approximation.
    Xp (1D array): List of points at which to evaluate the Taylor approximation.
    C (2D array): Coefficients of the Taylor series expansion.

    Returns:
    T2_taylor (1D array): An array of Taylor approximations of the squared traveltime at each point in Xp.
    """
    T2_taylor = np.zeros(len(Xp))
    for i in range(len(Xp)):
        T2_taylor[i] = sum(C[n, 0]*(Xp[i]**(2*n)) for n in range(approximation_order+1))
    return T2_taylor

#Compute Taylor Approximation of squared traveltime for nth order 
Taylor_approximation_order = 1
T2_taylor = T_squared_taylor(Taylor_approximation_order, Xp, C)

### Pade [n, n]th Order Approximation of Squared Two-Way Traveltime Offset Relation

In [164]:
def compute_Pade_Coeffecients(n, C):
    """
    Compute the coefficients for the Pade[n, n] order approximation. Pade approximant, a rational function
    is a ratio of two polynomial expansions with P0, P1, P2,..., Pn coefecients in numerator and Q1, Q2,..., Qn 
    coeffecients in denomenator.

    Parameters:
    n (int): The order of the Pade approximation.
    C (2D array): Coefficients of the Taylor series expansion.

    Returns:
    Qn (1D array): Coefficients of the numerator of the Pade approximation.
    Pn (1D array): Coefficients of the denominator of the Pade approximation.
    Cn (1D array array): Matrix of coefficients for the Pade approximation.
    b (1D array): Vector of coefficients for the Pade approximation.
    """
    # convert decimal objects to float64 for no execution error while using scipy
    # scipy.linalg is ot designed to handle dtype'object' elements
    Cn_taylor = np.array([np.float64(i) for i in C[:,0]])
    # compute Cn for pade approximation(eq.14)
    Cn = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            Cn[i, j] = Cn_taylor[i + j + 1]
    # compute b for pade approximation(eq.14)
    b = np.array(Cn_taylor[n+1:2*n+1]).reshape((n, 1))

    # Lu Decomposition of Cn
    P, L, U = scipy.linalg.lu(Cn)

    # solve for Qn, L*U*Qn = -p*b (eqn. 13)
    U_Qn = scipy.linalg.solve_triangular(L, np.dot(P, -b),lower=True)
    Qn = scipy.linalg.solve_triangular(U, U_Qn)
    Qn = Qn.flatten()  
    Qn = np.insert(Qn[::-1], 0, 1)   # it sets Q0 = 1 (Baker 1975; Aptekarev et al., 2011)
    Qn = np.array([Decimal(i) for i in Qn])
    # solve for Pn, pk = sum(Qi*Ck-i for i in range(0, k))
    Pn = np.zeros(n+1)
    for k in range(n+1):
        Pn[k] = sum(Qn[i]*C[k-i, 0] for i in range(k+1))
    
    Qn = np.array([Decimal(i) for i in Qn])
    Pn = np.array([Decimal(i) for i in Pn])
    return Qn, Pn, Cn, b

In [165]:
def T_squared_pade(Qn, Pn, Xp):
    """
    Compute the Pade approximation of the squared traveltimes for a set of offsets of a given velocity model.
     
    Parameters:
    Qn (1D Array): coefficients of the denominator polynomial
    Pn (2D Array): coefficients of the numerator polynomial
    Xp (2D Array): points at which to evaluate the Padé approximation

    Returns:
    T2_pade (array): T-squared Padé approximation evaluated at offsets in Xp array
    """
    T2_pade = np.zeros(len(Xp))
    for i in range(len(Xp)):
        num = sum(Pn[k]*Xp[i]**(2*k) for k in range(len(Pn)))
        den = sum(Qn[k]*Xp[i]**(2*k) for k in range(len(Qn)))
        T2_pade[i] = num/den
    return T2_pade

In [166]:
#Compute Pade Approximation of squared traveltime for [n, n]th order 
Pade_approximation_order = 3
Qn3, Pn3, Cn3, bn3 = compute_Pade_Coeffecients(Pade_approximation_order, C)
T2_pade33 = T_squared_pade(Qn3, Pn3, Xp)

Pade_approximation_order = 7
Qn7, Pn7, Cn7, bn7 = compute_Pade_Coeffecients(Pade_approximation_order, C)
T2_pade77 = T_squared_pade(Qn7, Pn7, Xp)

print(f"Cn3 :\n {Cn3}\n")
print(f"Qn3 :\n {Qn3}\n")
print(f"Pn3 :\n {Pn3}\n")

Cn3 :
 [[ 1.37287269e-01 -4.26579928e-04 -2.48615666e-07]
 [-4.26579928e-04 -2.48615666e-07  1.45152574e-08]
 [-2.48615666e-07  1.45152574e-08  1.61829883e-10]]

Qn3 :
 [Decimal('1')
 Decimal('-0.005010231672214942970355888718358983169309794902801513671875')
 Decimal('0.000087718995721034463165788341854067766689695417881011962890625')
 Decimal('1.57758862939066656056898000672072956973579493933357298374176025390625E-7')]

Pn3 :
 [Decimal('9.187755225624652410942871938459575176239013671875')
 Decimal('0.0912544866997028469857156096622929908335208892822265625')
 Decimal('-0.000308480290068194698029080402790214066044427454471588134765625')
 Decimal('0.00001538079977461751963476420390630750034688389860093593597412109375')]



#### Check pade coeffecients calculations are correct or not?!

In [167]:
# Create C0, C1, C2, C3,... variables e.g C[0, 0] is named C0,C[1, 0] is named C1 etc
for i in range(len(C[:, 0])):
    var_name = f'C{i}'
    locals()[var_name] = C[i, 0]

In [168]:
# Compare the Q1 with Song Q1
num = -C3**2*C4 + C2*C4**2 +C2*C3*C5 -C1*C4*C5 - C2**2*C6 +C1*C3*C6 
den = -C3**3 + 2*C2*C3*C4 -C1*C4**2 - C2**2*C5 + C1*C3*C5
Q1 = num/den
print("Song value, Computed value for Q1:", Q1, Qn3[1])

# Compare the Q2 with Song Q2
num = -C3*C4**3 + C3**2*C5 +C2*C4*C5 +C1*C5**2 +C2*C3*C6 - C1*C4*C6  
den = C3**3 - 2*C2*C3*C4 + C1*C4**2 + C2**2*C5 - C1*C3*C5
Q2 = num/den
print("Song value, Computed value for Q2:", Q2, Qn3[2])

# Compare the Q3 with Song Q3
num = -C4**3 + 2*C3*C4*C5 -C2*C5**2 - C3**2*C6 +C2*C4*C6
den = C3**3 - 2*C2*C3*C4 + C1*C4**2 + C2**2*C5 - C1*C3*C5
Q3 = num/den
print("Song value, Computed value for Q3:", Q3, Qn3[3],"\n")

# Compare the P0 with Song P0
print("Song value, Computed value for P0:",C0, Pn3[0])

# Compare the P1 with Song P1
num = -C1*C3**3 + 2*C1*C2*C3*C4 +C0*C3**2*C4 -C1**2*C4**2 - C0*C2*C4**2 -C1*C2**2*C5 + C1**2*C3*C5 - C0*C2*C3*C5 + C0*C1*C4*C5 + C0*C2**2*C6 - C0*C1*C3*C6
den = -C3**3 + 2*C2*C3*C4 -C1*C4**2 - C2**2*C5 + C1*C3*C5
P1 = num/den
print("Song value, Computed value for P1:", P1, Pn3[1])

# Compare the P2 with Song P2
num = C2*C3**3 - 2*C2**2*C3*C4 - C1*C3**2*C4 + 2*C1*C2*C4**2 + C0*C3*C4**2 + C3**2*C5 - C0*C3**2*C5 - C1**2*C4*C5 - C0*C2*C4*C5 + C0*C1*C5**2 - C1*C2**2*C6 + C1**2*C3*C6 + C0*C2*C3*C6 - C0*C1*C4*C6
den = C3**3 - 2*C2*C3*C4 + C1*C4**2 + C2**2*C5 - C1*C3*C5
P2 = num/den
print("Song value, Computed value for P2:", P2, Pn3[2])

Song value, Computed value for Q1: 0.0050102316722149429025018248400880 -0.005010231672214942970355888718358983169309794902801513671875
Song value, Computed value for Q2: 0.000055949402812024969870859129413237 0.000087718995721034463165788341854067766689695417881011962890625
Song value, Computed value for Q3: 1.5775886293906670764696124440657E-7 1.57758862939066656056898000672072956973579493933357298374176025390625E-7 

Song value, Computed value for P0: 9.1877552256246521641447349279194 9.187755225624652410942871938459575176239013671875
Song value, Computed value for P1: 0.091254486699702842174955019744013 0.0912544866997028469857156096622929908335208892822265625
Song value, Computed value for P2: -0.00010171369480860905923382914769152 -0.000308480290068194698029080402790214066044427454471588134765625


### Ghosh-Kumar 2nd Order Approximation of Squared Two-Way Traveltime Offset Relation

In [169]:
def T_squared_GhoshKumar(crit_angle, v, d, Xp):  
    """
    Compute the Ghosh Kumar approximation of the squared traveltimes for a set of offsets of a given velocity model.
    The offset in relation to critical angle divided by 1.2 is taken as the non-zero expansion point for the travel-time series.
    
    Parameters:
    crit_angle (float): critical angle
    v (1D array): Array containing the velocities of a horizontally isotropic layered Earth.
    d (1D array): Array containing the thicknesses of a horizontally isotropic layered Earth.
    Xp (1D array): offsets at which to evaluate the approximation

    Returns:
    Tgk (Decimal): Two-Way Traveltime for the non-zero offset(the expansion point)
    A (Decimal): nonzero-offset chosen for GK expansion
    T2_GhoshK (1D Array): 2nd order approximation of squared traveltime evaluated at each offsets in Xp
    """
    reference_angle = crit_angle/Decimal(1.1) 
    p = Decimal(math.sin(math.radians(reference_angle)))/v[0]
    delta_t = d/v
    # compute Tgk, offset A for nonzero-offset choosen for GK expansion 
    A = 2 * sum(((v[k]**2) * (delta_t[k]) * p/(1 - (v[k]**2) * (p**2)).sqrt()) for k in range(len(v)))  # eqn 4.1 , Taner and Kohler 
    TA = 2 * sum((delta_t[k]/(1 - (v[k]**2) * (p**2)).sqrt()) for k in range(len(v)))  # eqn 4.2 , Taner and Kohler 
    
    # compute the coefficients of the expansion
    # 0th order  (eqn 5.a, song paper)
    W0 = TA**2
    # 1st order  (eqn 5.a, song paper)
    W1 = 2*TA*p
    # 2nd order  (eqn 5.a, song paper)
    num = TA
    den = 2 * sum((d[i]*v[i]/(1 - (p**2) * (v[i]**2))**Decimal(3/2)) for i in range(len(v)))
    W2 = p**2 + num/den
    # 2nd order approximation of squared traveltime
    T2_GhoshK = np.zeros(len(Xp))
    for i in range(len(Xp)):
        T2_GhoshK[i] = W0 + W1 * (Xp[i]-A) + W2 * (Xp[i]-A)**2  # (eqn 5.a, song paper)

    return TA, A, T2_GhoshK

In [170]:
#Compute Ghosh Kumar Approximation of squared traveltime for 2nd order 
Tgk, A, T2_GhoshK = T_squared_GhoshKumar(crit_angle, v, d, Xp)
A, crit_angle/Decimal(1.2)
p = Decimal(math.sin(math.radians(34.8)))/v[0]

### Hybrid Taylor - Ghosh-Kumar(2nd-order) Approximation of Squared Two-Way Traveltime Offset Relation

In [171]:
def Hybrid_GhoshK(T2_GhoshK, T2_taylor, Tp):
    """
    Compute the Hybrid Ghosh-Kumar approximation of the squared traveltimes using Taylor and Ghosh-Kumar methods.
    The function evaluates which of the Taylor or Ghosh-Kumar approximations has a lower error and selects the one 
    with the lower error to create a hybrid approximation.

    Parameters:
    T2_GhoshK (1D array): Array containing the Ghosh-Kumar approximation of squared traveltimes.
    T2_taylor (1D array): Array containing the Taylor approximation of squared traveltimes.
    Tp (1D array): Array containing the traveltime values at different offsets.

    Returns:
    T2_Hybrid_GK (1D array): Hybrid approximation of squared traveltimes, choosing between Taylor and 
    Ghosh-Kumar approximations based on error evaluation.
    """
    T2_Taylor = np.array([Decimal(i) for i in T2_taylor])
    T2_GhoshK = np.array([Decimal(i) for i in T2_GhoshK])
    error_taylor = abs(Tp**2 - T2_Taylor)
    error_ghoshk = abs(Tp**2 - T2_GhoshK)
    for i in range(len(Tp)):
        if error_taylor[i] <= error_ghoshk[i]:
            T2_GhoshK[i] = T2_Taylor[i]
    T2_Hybrid_GK = T2_GhoshK
    return T2_Hybrid_GK
    
T2_Hybrid_GK = Hybrid_GhoshK(T2_GhoshK, T2_taylor, Tp)

### write traveltimes into a file

In [172]:
def save_result( incidence_angles, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_GhoshK, T2_Hybrid_GK, Model_Name):
    """
    Save the computed results of various traveltime approximations to a file.
    
    Parameters:
    incidence_angles (1D array): Array containing the incidence angles.
    d (1D array): Array containing the thicknesses of a horizontally isotropic layered Earth.
    Xp (1D array): Offsets at which the approximations are evaluated.
    Tp (1D array): Two-way traveltimes at different offsets.
    T2_taylor (1D array): Array containing the Taylor approximation of squared traveltimes.
    T2_pade33 (1D array): Array containing the Padé (3,3) approximation of squared traveltimes.
    T2_pade77 (1D array): Array containing the Padé (7,7) approximation of squared traveltimes.
    T2_GhoshK (1D array): Array containing the Ghosh-Kumar approximation of squared traveltimes.
    T2_Hybrid_GK (1D array): Array containing the Hybrid Ghosh-Kumar approximation of squared traveltimes.
    Model_Name (str): Name of the model, which is used to create the output filename.

    Returns:
    None
    """
    filename = f"{Model_Name}.dat"
    with open(filename, 'w') as file:
        # Write the header
        header = "{:<10} {:<35} {:<30} {:<40} {:<35} {:<35} {:<35} {:<35} {:<35}\n".format(
            "Index", "Incidence Angle", f"Xp/{sum(d):.3E}", "T2", "T2 Taylor", "T2 Pade33", "T2 Pade77", "T2 GhoshK", "T2_Hybrid_GK")
        file.write(header)
        file.write("-" * 290 + "\n")
        
        # Write the data rows
        for idx in range(len(Xp)):
            row = "{:<10} {:<16} {:<15} {:<15} {:<15} {:<15} {:<18}\n".format(
                idx, 
                f"{incidence_angles[idx]:.3E}", 
                f"{(Xp[idx]/sum(d)):.32E}", 
                f"{Tp[idx]**2:.32E}", 
                f"{T2_taylor[idx]:.32E}", 
                f"{T2_pade33[idx]:.32E}",
                f"{T2_pade77[idx]:.32E}\t"
                f"{T2_GhoshK[idx]:.32E}\t"
                f"{T2_Hybrid_GK[idx]:.32E}"
            )
            file.write(row)

save_result( incidence_angles, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_GhoshK, T2_Hybrid_GK, Model_Name)

### Plotting the squred travel times

In [173]:
# determine number of points for plotting parameters until negetive travel times
def plot_params(T2_taylor, T2_pade33, T2_pade77, T2_GhoshK):
    # Find the first negative index for each series
    indices = [
        next((i for i, num in enumerate(T2_taylor) if num < 0), len(T2_taylor)),
        next((i for i, num in enumerate(T2_pade33) if num < 0), len(T2_pade33)),
        next((i for i, num in enumerate(T2_pade77) if num < 0), len(T2_pade77)),
        next((i for i, num in enumerate(T2_GhoshK) if num < 0), len(T2_GhoshK))
    ]
    # Determine the minimum index where the first negative value occurs
    plot_upto = min(indices)
    # determine the reflector
    index = len(v)
    if index == 1:
        reflector_name = "1st Reflector"
    elif index == 2:
        reflector_name = "2nd Reflector"
    elif index == 3:
        reflector_name = "3rd Reflector"
    elif index >= 4:
        reflector_name = f"{index}th Reflector"

    return plot_upto, reflector_name

plot_upto, reflector_name = plot_params(T2_taylor, T2_pade33, T2_pade77, T2_GhoshK)

In [174]:
def sqared_traveltime_plot(m, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_GK):
    fig = go.Figure()
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=T2_taylor[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='2nd Taylor',
        line=dict(color='blue', width=2, dash='2,2,2,2'),
        #marker=dict(color='red', size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=T2_pade33[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_pade33],
        textposition='top center',
        name=f'Pade [3, 3]',
        line=dict(color='deeppink', width=2, dash= 'longdashdot'),
        #marker=dict(color='orange', size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=T2_pade77[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_pade77],
        textposition='top center',
        name=f'Pade [7, 7]',
        line=dict(color='red', width=2, dash='dot'),
        #marker=dict(color='red', size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=T2_GK[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_GK],
        textposition='top center',
        name=f'Ghosh_Kumar',
        line=dict(color='darkgreen', width=2, dash='dashdot'),
        #marker=dict(color='red', size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m]**2,
        mode='lines',
        text=[f'{d:.2f}' for d in Tp**2],
        textposition='top center',
        name=f'Analytical',
        line=dict(color='black', width=2, dash='solid'),
        #marker=dict(color='green', size=8)
    ))

    # Adding titles and labels
    fig.update_layout(
        title='Taylor series approximation and analytical squared travel time plot',
        title_font=dict(size=24, color='black'),
        xaxis_title='Normalized Offset Xp (Xp/H)',
        xaxis_title_font=dict(size=18, color='black'),
        yaxis_title='Squared Travel Time(sec²)',
        yaxis_title_font=dict(size=18, color='black'),
        font=dict(size=16),
        legend=dict(
            font=dict(color='black', size=12, family='Arial black, bold'),  
            bordercolor='black',
            borderwidth=1,
            x=0.005,
            y=1,
            xanchor='left',
            yanchor='top'),
        plot_bgcolor='white',
        xaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black')
    )

    # Adding a hover template
    fig.update_traces(hovertemplate='Position: %{x:.2f}<br>T_square: %{y:.8f}')
    # Show the interactive plot
    fig.show()

sqared_traveltime_plot(plot_upto, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_GhoshK)

In [175]:
def traveltime_error_plot(m, d, Xp, Tp, T2_taylor, T2_pade33,T2_pade77, T2_GK, reflector_name):
    T2_taylor = np.array([Decimal(i).sqrt() for i in T2_taylor[:m]])
    T2_pade33 = np.array([Decimal(i).sqrt() for i in T2_pade33[:m]])
    T2_pade77 = np.array([Decimal(i).sqrt() for i in T2_pade77[:m]])
    T2_GK = np.array([Decimal(i).sqrt() for i in T2_GK[:m]])
    # compute difference
    error_taylor = Tp[:m] - T2_taylor[:m]
    error_pade33 = Tp[:m] - T2_pade33[:m]
    error_pade77 = Tp[:m] - T2_pade77[:m]
    error_GK = Tp[:m] - T2_GK[:m] 
    # compute reflector depths
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'cyan', 'magenta', 'brown', 'deeppink', 'darkgreen', 'deepblue', 'magenta']
    
    Xp_normalized = [x / Decimal(np.cumsum(d)[-1]) for x in Xp]

    fig = go.Figure()
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp_normalized[:m],
        y=error_taylor,
        mode='lines',
        text=[f'{d:.2f}' for d in error_taylor],
        textposition='top center',
        name=f'2nd Taylor',
        line=dict(color='black', width=2, dash='2,2,2,2'),
        marker=dict(color=colors[2], size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp_normalized[:m],
        y=error_pade33,
        mode='lines',
        text=[f'{d:.2f}' for d in error_pade33],
        textposition='top center',
        name=f'Pade [3, 3]',
        line=dict(color='black', width=2, dash='longdashdot'),
        marker=dict(color=colors[3], size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp_normalized[:m],
        y=error_pade77,
        mode='lines',
        text=[f'{d:.2f}' for d in error_pade77],
        textposition='top center',
        name=f'Pade [7, 7]',
        line=dict(color='black', width=2, dash='dot'),
        marker=dict(color=colors[6], size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp_normalized[:m],
        y=error_GK,
        mode='lines',
        text=[f'{d:.2f}' for d in error_GK],
        textposition='top center',
        name=f'Ghosh_Kumar',
        line=dict(color='black', width=2, dash='dashdot'),
        marker=dict(color=colors[(5) % len(colors)], size=8)
    ))
    fig.add_trace(go.Scatter(
        x=np.arange(0, 9),
        y=np.zeros((9,)),
        mode='lines',
        text=[f'{d:.2f}' for d in error_GK],
        textposition='top center',
        name=f'zero error',
        line=dict(color='black', width=2, dash='dash'),
        marker=dict(color=colors[(5) % len(colors)], size=8)
    ))
    # Adding titles and labels
    fig.update_layout(
        #title=f'Travel Time Error vs. Normalized offset { Model_Name}',
        title_font=dict(size=24, color='black'),
        xaxis_title=f'Offset/Depth (Xp/{(np.cumsum(d)[-1]):.3f})',
        xaxis_title_font=dict(size=18, color='black'),
        yaxis_title=f'{reflector_name}(msec)',
        yaxis_title_font=dict(size=18, color='black'),
        font=dict(size=16),
        legend=dict(
            font=dict(color='black', size=12, family='Arial black, bold'),  
            bordercolor='black',
            borderwidth=1,
            x=0.005,
            y=1,
            xanchor='left',
            yanchor='top'),
        plot_bgcolor='white',
        xaxis=dict(
            gridcolor='lightgrey', 
            showline=True, 
            linewidth=1, 
            linecolor='black', 
            range = [0,8], 
            tickfont = dict(color='black', size=12, family=('Arial black, bold'))
            ),
        yaxis=dict(
            gridcolor='lightgrey', 
            showline=True, 
            linewidth=1, 
            tickvals=[-0.01, -0.005, 0, 0.005, 0.01],
            ticktext=['-10ms', '5ms','0ms', '5ms', '10ms'],
            tickfont = dict(color='black', size=12, family=('Arial black, bold')), 
            linecolor='black', 
            range = [-0.01, 0.01]
            )
    )

    # Adding a hover template
    fig.update_traces(hovertemplate='Position: %{x:.2f}<br>Error: %{y:.8f}')
    fig.update_layout(height=500,  width=1600)
    fig.write_image(f"{Model_Name}_{reflector_name}_GK.png")
    # Show the interactive plot
    fig.show()

traveltime_error_plot(plot_upto, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_GhoshK, reflector_name)

In [176]:
def TravelTime_Error_Plots_with_HybridGK(m, crit_angle, n, v, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_Hybrid_GK, reflector_name):

    T_Taylor = np.array([Decimal(i).sqrt() for i in T2_taylor[:m]])
    T_Pade33 = np.array([Decimal(i).sqrt() for i in T2_pade33[:m]])
    T_Pade77 = np.array([Decimal(i).sqrt() for i in T2_pade77[:m]])
    T_Hyb_GK = np.array([Decimal(i).sqrt() for i in T2_Hybrid_GK[:m]])
    
    fig = go.Figure()
    # Adding the line plot
    colors = ['blue', 'darkgreen', 'red', 'purple', 'orange', 'darksalmon', 'magenta', 'lawngreen', 'hotpink', 'darkgoldenrod', 'deepblue', 'magenta']
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m] - T_Taylor,
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='2nd Taylor',
        line=dict(color='black', width=2, dash='2,2,2,2'),
        #marker=dict(color='red', size=2)
    ))
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m] - T_Pade33,
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='Pade33',
        line=dict(color='black', width=2, dash='longdashdot'),
        #marker=dict(color='red', size=2)
    ))
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m] - T_Pade77[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='Pade77',
        line=dict(color='black', width=2, dash='dot'),
        #marker=dict(color='red', size=2)
    ))
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m] - T_Hyb_GK,
        mode='lines',
        text=[f'{d:.2f}' for d in T_Hyb_GK],
        textposition='top center',
        name=f'Hybrid GhoshKumar',
        line=dict(color='black', width=2, dash='dashdot'),
        #marker=dict(color=colors[key], size=2)
    ))
    fig.add_trace(go.Scatter(
        x=np.arange(0, 9),
        y=np.zeros((9,)),
        mode='lines',
        text=[f"zero error"],
        textposition='top center',
        name='Zero Error',
        textfont=dict(color='red'),
        line=dict(color='black', width=2, dash='dash'),
        #marker=dict(color='red', size=2)
    ))
    # Adding titles and labels
    fig.update_layout(
        #title='Ghosh Kumar Travel time errors for asymptotic approximations about non-zero offsets',
        title_font=dict(size=24, color='black'),
        xaxis_title='Normalized Offset Xp (Xp/H)',
        xaxis_title_font=dict(size=18, color='black'),
        yaxis_title=f'{reflector_name}(msec)',
        yaxis_title_font=dict(size=18, color='black'),
        font=dict(size=16),
        legend=dict(
            font=dict(color='black', size=12, family='Arial black, bold'),  
            bordercolor='black',
            borderwidth=1,
            x=0.005,
            y=1,
            xanchor='left',
            yanchor='top'),
        plot_bgcolor='white',
        xaxis=dict(
            gridcolor='lightgrey', 
            showline=True, 
            linewidth=1, 
            linecolor='black', 
            range = [0,8], 
            tickfont = dict(color='black', size=12, family=('Arial black, bold'))
            ),
        yaxis=dict(
            gridcolor='lightgrey', 
            showline=True, 
            linewidth=1, 
            tickvals=[-0.01, -0.005, 0, 0.005, 0.01],
            ticktext=['-10ms', '5ms','0ms', '5ms', '10ms'],
            tickfont = dict(color='black', size=12, family=('Arial black, bold')), 
            linecolor='black', 
            range = [-0.01, 0.01]
            )
    )

    # Adding a hover template
    fig.update_traces(hovertemplate='Position: %{x:.2f}<br>T_square: %{y:.8f}')
    #fig.write_image(f"{Model_Name}_{reflector_name}_Hyb_GK.png")
    # Show the interactive plot
    fig.show()

In [177]:
TravelTime_Error_Plots_with_HybridGK(plot_upto, crit_angle, [1.2], v, d, Xp, Tp, T2_taylor, T2_pade33, T2_pade77, T2_Hybrid_GK, reflector_name)

### RMS Error

In [178]:
def rms_error(start_point, end_point, Tp, T2):
    """
    Calculate the Root Mean Square (RMS) error between the actual traveltime (Tp) and 
    the approximated traveltime (T) over a specified range.

    Parameters:
    start_point (int): The starting index for the calculation range.
    end_point (int): The ending index for the calculation range.
    Tp (1D array): Array of actual two-way traveltimes.
    T2 (1D array): Array of approximated squared two-way traveltimes.

    Returns:
    rms_error (Decimal): The RMS error computed over the specified range.
    """
    Tp = Tp[start_point:end_point]
    T = T2[start_point:end_point]
    summation = Decimal(0.0)
    for i in range(len(Tp)):
        summation += (Tp[i] - Decimal(T[i]).sqrt())**2
    rms_error = summation.sqrt()
    return rms_error

In [179]:
# compute the error for the whole values(1, X/H)
start_point = 0
end_point = plot_upto
rms_taylor = rms_error(start_point, end_point, Tp, T2_taylor)
rms_Pade33 = rms_error(start_point, end_point, Tp, T2_pade33)
rms_Pade77 = rms_error(start_point, end_point, Tp, T2_pade77)
rms_GhoshK = rms_error(start_point, end_point, Tp, T2_GhoshK)
rms_Hyb_GK = rms_error(start_point, end_point, Tp, T2_Hybrid_GK)

print(f"Model_Name : {Model_Name}\n"
      f"rms_error_taylor = {rms_taylor:.3E}\n"
      f"rms_error_Pade33 = {rms_Pade33:.3E}\n"
      f"rms_error_Pade77 = {rms_Pade77:.3E}\n"
      f"rms_error_GhoshK = {rms_GhoshK:.3E}\n"
      f"rms_error_Hyb_GK = {rms_Hyb_GK:.3E}")


Model_Name : Cooper_basin
rms_error_taylor = 4.722E+1
rms_error_Pade33 = 3.056E+1
rms_error_Pade77 = 3.022E+1
rms_error_GhoshK = 1.572E+1
rms_error_Hyb_GK = 1.569E+1


### ........................................................................Additional Portion................................................................................

##### Compute T2_GhoshKumar for several choices of non-zero points of expansion

In [180]:
def T_squared_GhoshK(crit_angle, m, v, d, Xp): 
    """
    Compute the Ghosh-Kumar approximation of the squared traveltimes for a set of offsets for a given velocity model.

    Parameters:
    crit_angle (float): Critical angle for the velocity model.
    m (1D array): Array of factors for dividing the critical angle to get the reference angle for the non-zero expansion point.
    v (1D array): Array of velocities for each layer in a horizontally isotropic layered Earth.
    d (1D array): Array of thicknesses for each layer in a horizontally isotropic layered Earth.
    Xp (1D array): Array of offsets at which the approximation is evaluated.

    Returns:
    factors (1D array): The input factors array.
    reference_angle (dict): Dictionary containing the reference angle for each factor.
    T2_GhoshKumar (dict): Dictionary containing the Ghosh-Kumar squared traveltime approximation for each factor.
    """
    factors = np.array([i for i in m])
    T2_GhoshKumar = {} 
    reference_angle = {}
    for i in range(len(factors)):
        reference_angle[i] = crit_angle/Decimal(factors[i]) 
        p = Decimal(math.sin(math.radians(reference_angle[i])))/v[0]
        delta_t = d/v
        # compute Tgk, offset A for nonzero-offset choosen for GK expansion 
        Tgk = 2 * sum(delta_t[k]/(1 - (v[k]**2) * (p**2)).sqrt() for k in range(len(v)))
        A = 2 * sum((v[k]**2) * (delta_t[k]) * p/(1 - (v[k]**2) * (p**2)).sqrt() for k in range(len(v)))
        # compute the coefficients of the expansion
        # 0th order
        W0 = Tgk**2
        # 1st order
        W1 = 2*Tgk*p
        # 2nd order
        num = Tgk
        den = 2 * sum((d[i]*v[i])/(1 - (p**2) * (v[i]**2))**Decimal(3/2) for i in range(len(v)))
        W2 = p**2 + num/den
        # 2nd order approximation of squared traveltime
        T2_GhoshK = np.zeros(len(Xp))
        for j in range(len(Xp)):
            T2_GhoshK[j] = W0 + W1 * (Xp[j]-A) + W2 * (Xp[j]-A)**2
        T2_GhoshKumar[i] = T2_GhoshK

    return factors, reference_angle, T2_GhoshKumar

factors, reference_angle, T2_GhoshKumar = T_squared_GhoshK(crit_angle, [1.1, 1.15, 1.2, 1.25], v, d, Xp)

In [181]:
def many_sqared_GK_traveltime_plots(m, crit_angle, n, v, d, Xp, Tp):
    factors, reference_angle, T2_GhoshKumar = T_squared_GhoshK(crit_angle, n, v, d, Xp)
    fig = go.Figure()
    colors = ['blue', 'darkseagreen', 'red', 'purple', 'orange', 'darksalmon', 'magenta', 'lawngreen', 'hotpink', 'darkgoldenrod', 'deepblue', 'magenta']
    # Adding the line plot
    for key, values in T2_GhoshKumar.items():
        print(factors[key])
        fig.add_trace(go.Scatter(
            x=Xp[:m]/np.cumsum(d)[-1],
            y=T2_GhoshKumar[key][:m],
            mode='lines',
            text=[f'{d:.2f}' for d in T2_GhoshKumar[key]],
            textposition='top center',
            name=f'factors : {factors[key]:.3f} incidence angle : {reference_angle[key]:.3f} ',
            line=dict(color=colors[key], width=2, dash='dashdot'),
            #marker=dict(color='red', size=8)
        ))
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=T2_taylor[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='Taylor Approximation of Squared Two-Way TravelTime ',
        line=dict(color='red', width=2, dash='10px,5px,2px,5px'),
        #marker=dict(color='red', size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m]**2,
        mode='lines',
        text=[f'{d:.2f}' for d in Tp**2],
        textposition='top center',
        name='Analytical Squared TWT',
        line=dict(color='black', width=2, dash='solid'),
        #marker=dict(color='green', size=8)
    ))

    # Adding titles and labels
    fig.update_layout(
        title='Ghosh Kumar asymptotic approximation for non-zero offset',
        title_font=dict(size=24, color='black'),
        xaxis_title='Normalized Offset Xp (Xp/H)',
        xaxis_title_font=dict(size=18, color='black'),
        yaxis_title='Squared Travel Time(sec²)',
        yaxis_title_font=dict(size=18, color='black'),
        font=dict(size=16),
        legend=dict(font=dict(size=16)),
        plot_bgcolor='white',
        xaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black')
    )

    # Adding a hover template
    fig.update_traces(hovertemplate='Position: %{x:.2f}<br>T_square: %{y:.8f}')
    # Show the interactive plot
    fig.show()

#many_sqared_GK_traveltime_plots(plot_upto, crit_angle, [1.1, 1.15, 1.2, 1.25], v, d, Xp, Tp)

In [182]:
def many_sqared_GK_traveltime_error_plots(m, crit_angle, n, v, d, Xp, Tp, T2_taylor):
    factors, reference_angle, T2_GhoshKumar = T_squared_GhoshK(crit_angle, n, v, d, Xp)
    T2_Taylor = np.array([Decimal(i) for i in T2_taylor])
    fig = go.Figure()
    # Adding the line plot
    colors = ['blue', 'darkgreen', 'red', 'purple', 'orange', 'darksalmon', 'magenta', 'lawngreen', 'hotpink', 'darkgoldenrod', 'deepblue', 'magenta']
    for key, values in T2_GhoshKumar.items():
        T2_Gk = np.array([Decimal(i) for i in T2_GhoshKumar[key]])
        error = Tp**2 - T2_Gk
        fig.add_trace(go.Scatter(
            x=Xp[:m]/np.cumsum(d)[-1],
            y=error[:m],
            mode='lines',
            text=[f'{d:.2f}' for d in T2_GhoshKumar[key]],
            textposition='top center',
            name=f'factors : {factors[key]:.3f}, incidence angle : {reference_angle[key]:.3f} ',
            line=dict(color=colors[key], width=2, dash='dashdot'),
            #marker=dict(color=colors[key], size=2)
        ))
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m]**2 - T2_Taylor[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='Taylor Approximation of Squared Two-Way TravelTime ',
        line=dict(color='red', width=2, dash='10px,5px,2px,5px'),
        #marker=dict(color='red', size=2)
    ))
    # Adding titles and labels
    fig.update_layout(
        title='Ghosh Kumar Travel time errors for asymptotic approximations about non-zero offsets',
        title_font=dict(size=24, color='black'),
        xaxis_title='Normalized Offset Xp (Xp/H)',
        xaxis_title_font=dict(size=18, color='black'),
        yaxis_title='Squared Travel Time(sec²)',
        yaxis_title_font=dict(size=18, color='black'),
        font=dict(size=16),
        legend=dict(font=dict(size=16)),
        plot_bgcolor='white',
        xaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black')
    )

    # Adding a hover template
    fig.update_traces(hovertemplate='Position: %{x:.2f}<br>T_square: %{y:.8f}')
    # Show the interactive plot
    fig.show()

#many_sqared_GK_traveltime_error_plots(plot_upto, crit_angle, [1.1, 1.15, 1.2, 1.25], v, d, Xp, Tp, T2_taylor)

##### Compute Hybrid Taylor-GhoshKumar for several choices of non-zero points(offsets) of expansion

In [183]:
def Hybrid_GhoshK_offsets(T2_GhoshKumar, T2_taylor, Tp):
    T2_Hybrid_GK_offsets = {}
    for key, value in T2_GhoshKumar.items():
        T2_Taylor = np.array([Decimal(i) for i in T2_taylor])
        T2_GhoshK = np.array([Decimal(i) for i in value])
        error_taylor = abs(Tp**2 - T2_Taylor)
        error_ghoshk = abs(Tp**2 - T2_GhoshK)
        T2_Hybrid_GK_offsets[key] = np.zeros(len(Tp), dtype=Decimal)  
        for i in range(len(Tp)):
            if error_taylor[i] <= error_ghoshk[i]:
                T2_Hybrid_GK_offsets[key][i] = T2_Taylor[i]
            else:
                T2_Hybrid_GK_offsets[key][i] = T2_GhoshK[i]
        
    return T2_Hybrid_GK_offsets

In [184]:
def many_sqared_Hybrid_GK_traveltime_plot_offsets(m, crit_angle, n, v, d, Xp, Tp, T2_taylor):
    factors, reference_angle, T2_GhoshKumar = T_squared_GhoshK(crit_angle, n, v, d, Xp)
    T2_Hybrid_GK_offsets = Hybrid_GhoshK_offsets(T2_GhoshKumar, T2_taylor, Tp)
    fig = go.Figure()
    colors = ['blue', 'darkseagreen', 'red', 'purple', 'orange', 'darksalmon', 'magenta', 'lawngreen', 'hotpink', 'darkgoldenrod', 'deepblue', 'magenta']
    # Adding the line plot
    for key, values in T2_GhoshKumar.items():
        print(factors[key])
        fig.add_trace(go.Scatter(
            x=Xp[:m]/np.cumsum(d)[-1],
            y=T2_Hybrid_GK_offsets[key][:m],
            mode='lines',
            text=[f'{d:.2f}' for d in T2_GhoshKumar[key]],
            textposition='top center',
            name=f'factors : {factors[key]:.3f} incidence angle : {reference_angle[key]:.3f} ',
            line=dict(color=colors[key], width=2, dash='solid'),
            #marker=dict(color='red', size=8)
        ))
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=T2_taylor[:m],
        mode='lines',
        text=[f'{d:.2f}' for d in T2_taylor],
        textposition='top center',
        name='Taylor Approximation of Squared Two-Way TravelTime ',
        line=dict(color='red', width=2, dash='solid'),
        #marker=dict(color='red', size=8)
    ))
    # Adding the line plot
    fig.add_trace(go.Scatter(
        x=Xp[:m]/np.cumsum(d)[-1],
        y=Tp[:m]**2,
        mode='lines',
        text=[f'{d:.2f}' for d in Tp**2],
        textposition='top center',
        name='Analytical Squared TWT',
        line=dict(color='black', width=2, dash='solid'),
        #marker=dict(color='green', size=8)
    ))

    # Adding titles and labels
    fig.update_layout(
        title='Hybrid Ghosh Kumar asymptotic approximations for non-zero offsets',
        title_font=dict(size=24, color='black'),
        xaxis_title='Normalized Offset Xp (Xp/H)',
        xaxis_title_font=dict(size=18, color='black'),
        yaxis_title='Squared Travel Time(sec²)',
        yaxis_title_font=dict(size=18, color='black'),
        font=dict(size=16),
        legend=dict(font=dict(size=16)),
        plot_bgcolor='white',
        xaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(gridcolor='lightgrey', showline=True, linewidth=1, linecolor='black')
    )

    # Adding a hover template
    fig.update_traces(hovertemplate='Position: %{x:.2f}<br>T_square: %{y:.8f}')
    # Show the interactive plot
    fig.show()

# many_sqared_Hybrid_GK_traveltime_plot_offsets(plot_upto, crit_angle, [1.1, 1.15, 1.2, 1.25], v, d, Xp, Tp, T2_taylor)